# Introduction
Les données économiques retenues dans le cadre de ce travail nous proviennent du site de la FAO et celui de la BCEAO. En raison des limites liées au chargement direct via API, elles ont été téléchargées en local au préalable.Elles ont ensuite été intégrées via Unity Catalog sous forme de table Delta, puis nettoyées et typées pour l’analyse économico-climatique.
La manière de charger ici consiste à aller au niveau de l'onglet home puis upload files, selectionner les fichiers à charger. Après cela, elles sont immédiatement tansformées en tables utilisable par Apache Spark. Les données chargéees sont brutes.

Les données sont annuelles et concernent les dépenses en agriculture annuelle, le PIB brut annuel, les rendements agricoles annuels et la production annuelle.

In [0]:
# Chargeons les données agricoles notemment les rendements agricoles, la production et la surface cultivée
from pyspark.sql import functions as F

df_agri_raw = spark.table("workspace.default.donnees_agricoles")
print("Volume initial :", df_agri_raw.count())
display(df_agri_raw.limit(5))


Volume initial : 2549


Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
QCL,Crops and livestock products,686,Senegal,5312,Area harvested,1211.0,Asparagus,2005,2005,ha,null,M,Missing value; data cannot exist,null
QCL,Crops and livestock products,686,Senegal,5510,Production,1211.0,Asparagus,2005,2005,t,null,M,Missing value; data cannot exist,null
QCL,Crops and livestock products,686,Senegal,5312,Area harvested,1211.0,Asparagus,2006,2006,ha,null,M,Missing value; data cannot exist,null
QCL,Crops and livestock products,686,Senegal,5510,Production,1211.0,Asparagus,2006,2006,t,null,M,Missing value; data cannot exist,null
QCL,Crops and livestock products,686,Senegal,5312,Area harvested,1211.0,Asparagus,2007,2007,ha,null,M,Missing value; data cannot exist,null


Le volume initial a 2549 observations

In [0]:
# On retient les variables qui nous intéressent
df_agri_sn = (
    df_agri_raw
    .withColumn("Year", F.col("Year").cast("int"))
    .withColumn("Value", F.col("Value").cast("double"))
)

In [0]:
# Vérification des valeurs manquantes
df_agri_sn.select(
    F.count(F.when(F.col("Value").isNull(), 1)).alias("Valeurs_manquantes")
).show()


+------------------+
|Valeurs_manquantes|
+------------------+
|               494|
+------------------+



On a 494 valeurs manquantes. Dans la suite, on supprime les valeurs manquantes parceque leur suppression n'a pas un impact significative.

In [0]:
# Suppression des valeurs manquantes
df_agri_clean = df_agri_sn.filter(F.col("Value").isNotNull())

print("Après suppression des NA :", df_agri_clean.count())


Après suppression des NA : 2055


Après suppression, on 2055 observations

In [0]:
# Gardons les variables importantes
df_agri_final = df_agri_clean.select(
    "Year",
    "Item",
    "Element",
    "Unit",
    "Value"
)


In [0]:
# On sauvegarde la base
df_agri_final.write.mode("overwrite").saveAsTable(
    "workspace.default.data_economics_clean"
)


In [0]:
# On affiche pour vérifier si la sauvegarde a été faite
spark.table("workspace.default.data_economics_clean").printSchema()
display(
    spark.table("workspace.default.data_economics_clean")
    .orderBy("Year")
    .limit(10)
)


root
 |-- Year: integer (nullable = true)
 |-- Item: string (nullable = true)
 |-- Element: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Value: double (nullable = true)



Year,Item,Element,Unit,Value
2005,Carrots and turnips,Area harvested,ha,247.0
2005,Cabbages,Production,t,25350.0
2005,Cabbages,Area harvested,ha,1014.0
2005,Carrots and turnips,Production,t,3700.0
2005,Bananas,Yield,kg/ha,28026.6
2005,Cabbages,Yield,kg/ha,25000.0
2005,Bananas,Area harvested,ha,939.0
2005,Bananas,Production,t,26317.0
2005,Carrots and turnips,Yield,kg/ha,14979.8
2005,"Cashew nuts, in shell",Area harvested,ha,17851.0


Les données précédentes sont sur la période de 2005 à 2023

Chargeons à présent les données sur le PIB et les dépenses en agriculture

In [0]:
# Données sur le PIB Brut
from pyspark.sql import functions as F

df_pib_raw = spark.read.table("pib_brut")
df_pib_raw.printSchema()


root
 |-- Domain Code: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- Area Code (M49): long (nullable = true)
 |-- Area: string (nullable = true)
 |-- Element Code: long (nullable = true)
 |-- Element: string (nullable = true)
 |-- Item Code: long (nullable = true)
 |-- Item: string (nullable = true)
 |-- Year Code: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Flag Description: string (nullable = true)
 |-- Note: string (nullable = true)



In [0]:
# Retenons les variables utiles 
df_pib_clean = (
    df_pib_raw
    .select(
        F.col("Year").cast("int"),
        F.col("Area"),
        F.col("Item"),
        F.col("Element"),
        F.col("Unit"),
        F.col("Value").cast("double").alias("pib_value")
    )
    .filter(F.col("Area") == "Senegal")
)


In [0]:
# Sauvegardond la base
(df_pib_clean
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("pib_clean")
)

Le pib brut va de 2005 à 2023

Passons à la gestions des dépenses agricoles

In [0]:
# Chargeons les dépenses en agriculture
df_dep_raw = spark.read.table("depenses_agricoles")
df_dep_raw.printSchema()


root
 |-- Domain Code: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- Area Code (M49): long (nullable = true)
 |-- Area: string (nullable = true)
 |-- Element Code: long (nullable = true)
 |-- Element: string (nullable = true)
 |-- Item Code: long (nullable = true)
 |-- Item: string (nullable = true)
 |-- Year Code: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Value: long (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Flag Description: string (nullable = true)
 |-- Note: string (nullable = true)



In [0]:
# On garde les variables utiles
df_dep_clean = (
    df_dep_raw
    .select(
        F.col("Year").cast("int"),
        F.col("Area"),
        F.col("Item"),
        F.col("Element"),
        F.col("Unit"),
        F.col("Value").cast("double").alias("depense_agri")
    )
    .filter(F.col("Area") == "Senegal")
)


In [0]:
# On sauvegarde la base
(df_dep_clean
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("depenses_agricoles_clean")
)


Les données réelles disponibles ici vont de  2017 à 2024

In [0]:
# On merge les bases du pib brut  et des dépenses en agriculture
df_pib = spark.read.table("pib_clean")
df_dep = spark.read.table("depenses_agricoles_clean")

df_macro = (
    df_pib.join(df_dep, on="Year", how="left")
          .orderBy("Year")
)

display(df_macro)


Year,Area,Item,Element,Unit,pib_value,Area,Item,Element,Unit,depense_agri
2005,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,5804604.948747,null,null,null,null,null
2006,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,6111292.249282,null,null,null,null,null
2007,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,6698104.789508,null,null,null,null,null
2008,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,7516880.06491,null,null,null,null,null
2009,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,7593295.291504,null,null,null,null,null
2010,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,7976730.0,null,null,null,null,null
2011,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,8394960.0,null,null,null,null,null
2012,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,9016870.0,null,null,null,null,null
2013,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,9343920.0,null,null,null,null,null
2014,Senegal,Gross Domestic Product,Value Standard Local Currency,million SLC,9775039.0,null,null,null,null,null


In [0]:
# On merge toutes les bases qui traduisent l'économie
df_main = spark.read.table("data_economics_clean")
df_macro = spark.read.table("macro_agri_clean")

df_final = (
    df_main.join(df_macro, on="Year", how="left")
)

Les indicateurs de performance agricole ont été complétés par des variables macroéconomiques, notamment les dépenses publiques agricoles et le PIB annuel, agrégées chaque année pour garantir la comparabilité avec les données climatiques et de production.

In [0]:
# Rechargeons les données pour les merger
df_pib = spark.read.table("pib_brut")
df_dep = spark.read.table("depenses_agricoles")

Afin de merger les bases, on va recharger les tables en donnant des noms significatifs aux variables

In [0]:
# base pib 
df_pib_sel = (
    df_pib
    .select(
        F.col("Area"),
        F.col("Item").alias("pib_item"),
        F.col("Year"),
        F.col("Value").alias("pib_value"),
        F.col("Unit").alias("pib_unit")
    )
)


In [0]:
# base dépenses
df_dep_sel = (
    df_dep
    .select(
        F.col("Area"),
        F.col("Item").alias("dep_item"),
        F.col("Year"),
        F.col("Value").alias("dep_agri_value"),
        F.col("Unit").alias("dep_agri_unit")
    )
)


In [0]:
#merge des deux 
df_economics = (
    df_pib_sel
    .join(
        df_dep_sel,
        on=["Area", "Year"],
        how="left"
    )
)


In [0]:
# on affiche
display(df_economics.limit(10))

Area,Year,pib_item,pib_value,pib_unit,dep_item,dep_agri_value,dep_agri_unit
Senegal,2005,Gross Domestic Product,5804604.948747,million SLC,null,null,null
Senegal,2006,Gross Domestic Product,6111292.249282,million SLC,null,null,null
Senegal,2007,Gross Domestic Product,6698104.789508,million SLC,null,null,null
Senegal,2008,Gross Domestic Product,7516880.06491,million SLC,null,null,null
Senegal,2009,Gross Domestic Product,7593295.291504,million SLC,null,null,null
Senegal,2010,Gross Domestic Product,7976730.0,million SLC,null,null,null
Senegal,2011,Gross Domestic Product,8394960.0,million SLC,null,null,null
Senegal,2012,Gross Domestic Product,9016870.0,million SLC,null,null,null
Senegal,2013,Gross Domestic Product,9343920.0,million SLC,null,null,null
Senegal,2014,Gross Domestic Product,9775039.0,million SLC,null,null,null


In [0]:
# on charge la base de données sur les rendements agricoles
df_item_climate = spark.read.table("data_economics_clean")

In [0]:
# on la fusionne à la base qui contient le pib et les dépenses en agriculture
df_final_all = (
    df_item_climate
    .join(
        df_economics,
        on= "Year",
        how="left"
    )
)


In [0]:
# on affiche le résultat
df_final_all.printSchema()
display(df_final_all.orderBy("Year").limit(20))


root
 |-- Year: integer (nullable = true)
 |-- Item: string (nullable = true)
 |-- Element: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- Area: string (nullable = true)
 |-- pib_item: string (nullable = true)
 |-- pib_value: double (nullable = true)
 |-- pib_unit: string (nullable = true)
 |-- dep_item: string (nullable = true)
 |-- dep_agri_value: long (nullable = true)
 |-- dep_agri_unit: string (nullable = true)



Year,Item,Element,Unit,Value,Area,pib_item,pib_value,pib_unit,dep_item,dep_agri_value,dep_agri_unit
2005,"Chillies and peppers, dry (Capsicum spp., Pimenta spp.), raw",Yield,kg/ha,8008.3,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,"Cashew nuts, in shell",Production,t,5448.15,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,Cabbages,Area harvested,ha,1014.0,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,Bananas,Production,t,26317.0,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,Bananas,Yield,kg/ha,28026.6,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,Carrots and turnips,Yield,kg/ha,14979.8,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,Carrots and turnips,Area harvested,ha,247.0,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,Cabbages,Yield,kg/ha,25000.0,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,"Chillies and peppers, dry (Capsicum spp., Pimenta spp.), raw",Production,t,4805.0,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null
2005,Carrots and turnips,Production,t,3700.0,Senegal,Gross Domestic Product,5804604.948747,million SLC,null,null,null


In [0]:
# nombre total d'observation 
nb_lignes = df_final_all.count()
print(f"Nombre total de lignes : {nb_lignes}")


Nombre total de lignes : 2055


In [0]:
# nombre de colones
nb_colonnes = len(df_final_all.columns)
print(f"Nombre total de colonnes : {nb_colonnes}")


Nombre total de colonnes : 12


In [0]:
# résumé 
print("Colonnes :")
for c in df_final_all.columns:
    print("-", c)


Colonnes :
- Year
- Item
- Element
- Unit
- Value
- Area
- pib_item
- pib_value
- pib_unit
- dep_item
- dep_agri_value
- dep_agri_unit


## Base finale

In [0]:
# base finale 
(
    df_final_all
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("annual_economic_data")
)


In [0]:
# vérification
spark.read.table("annual_economic_data").limit(5).show()


+----+-------+--------------+-----+-------+-------+--------------------+--------------+-----------+--------+--------------+-------------+
|Year|   Item|       Element| Unit|  Value|   Area|            pib_item|     pib_value|   pib_unit|dep_item|dep_agri_value|dep_agri_unit|
+----+-------+--------------+-----+-------+-------+--------------------+--------------+-----------+--------+--------------+-------------+
|2005|Bananas|Area harvested|   ha|  939.0|Senegal|Gross Domestic Pr...|5804604.948747|million SLC|    NULL|          NULL|         NULL|
|2005|Bananas|         Yield|kg/ha|28026.6|Senegal|Gross Domestic Pr...|5804604.948747|million SLC|    NULL|          NULL|         NULL|
|2005|Bananas|    Production|    t|26317.0|Senegal|Gross Domestic Pr...|5804604.948747|million SLC|    NULL|          NULL|         NULL|
|2006|Bananas|Area harvested|   ha| 1222.0|Senegal|Gross Domestic Pr...|6111292.249282|million SLC|    NULL|          NULL|         NULL|
|2006|Bananas|         Yield|kg/ha

La base de données économique annuelle agrège les rendements agricoles,  production agricole, PIB brut ainsi que quelques cultures  pour le Sénégal sur la période de 2005 à 2025. Elle contient 2055 observations et 15 variables, couvrant la production agricole, les superficies cultivées, les rendements, les indicateurs climatiques moyens annuels ainsi que les variables économiques.